In [1]:
3+2

5

In [2]:
import xlwings

In [3]:
xlwings.sheets

Sheets([<Sheet [통합 문서1]Sheet1>])

In [5]:
xlwings.Range("A1")

<Range [통합 문서1]Sheet1!$A$1>

In [6]:
xlwings.Range("A1").expand()

<Range [통합 문서1]Sheet1!$A$1:$B$9>

In [9]:
total_rows = xlwings.Range("A1").expand().value
total_rows

[['message', 'browser'],
 ['첫 로그 ', 'chrome'],
 ['두 번째 로그', 'chrome'],
 ['세 번째 로그', 'chrome'],
 ['로그#4', 'chrome'],
 ['로그#5', 'chrome'],
 ['로그#6', 'chrome'],
 ['로그#7', 'chrome'],
 ['로그#8', 'chrome']]

In [10]:
column_names = total_rows[0]
column_names 

['message', 'browser']

In [17]:
for row in total_rows[1:]:
    row_dict = dict(zip(column_names, row))
    print(row_dict)

{'message': '첫 로그 ', 'browser': 'chrome'}
{'message': '두 번째 로그', 'browser': 'chrome'}
{'message': '세 번째 로그', 'browser': 'chrome'}
{'message': '로그#4', 'browser': 'chrome'}
{'message': '로그#5', 'browser': 'chrome'}
{'message': '로그#6', 'browser': 'chrome'}
{'message': '로그#7', 'browser': 'chrome'}
{'message': '로그#8', 'browser': 'chrome'}


In [19]:
# list comprehension
rows_date = [
    dict(zip(column_names, row))
    for row in total_rows[1:]
]
rows_date

[{'message': '첫 로그 ', 'browser': 'chrome'},
 {'message': '두 번째 로그', 'browser': 'chrome'},
 {'message': '세 번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'}]

# 정리

In [20]:
def get_rows_data():
    total_rows = xlwings.Range("A1").expand().value
    column_names = total_rows[0]
    rows_data = [
        dict(zip(column_names, row))
        for row in total_rows[1:]
    ]
    return rows_data

get_rows_data()

[{'message': '첫 로그 ', 'browser': 'chrome'},
 {'message': '두 번째 로그', 'browser': 'chrome'},
 {'message': '세 번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'}]

# 데이터베이스 접속

In [26]:
# 데이터베이스 접속
DATABASE = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "1234",
    "db": "mayproj11_db",
    "charset": "utf8",
}

In [27]:
import pymysql

connection = pymysql.connect(**DATABASE)
cursor = connection.cursor()
cursor.execute("SELECT * FROM log")
print(cursor.fetchall())
cursor.close()

()


# with 절

In [28]:
import pymysql
from contextlib import contextmanager

@contextmanager
def db_curser():
    connection = pymysql.connect(**DATABASE)
    cursor = connection.cursor()
    try:
        yield cursor
    finally:
        cursor.close()

In [30]:
with db_curser() as cursor:
    cursor.execute("SELECT * FROM log")
    print(cursor.fetchall())

()


In [40]:
column_data = {'message': '첫 로그', 'browser': 'chrome'}

with db_curser() as cursor:
    for row in get_rows_data():
        sql = "INSERT INTO LOG(message, brower) VALUES(%(message)s,%(browser)s)"
        cursor.execute(sql, ["첫 로그", "chrome"])

OperationalError: (1054, "Unknown column 'brower' in 'field list'")

In [35]:
get_rows_data()

[{'message': '첫 로그 ', 'browser': 'chrome'},
 {'message': '두 번째 로그', 'browser': 'chrome'},
 {'message': '세 번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'}]

In [59]:
column_data = {'message': '첫 로그', 'browser': 'chrome'}

with db_curser() as cursor:
    log_id = '1 or 1'
#     sql = "SELECT * FROM LOG WHERE id = " + str(log_id)
#     sql = "SELECT * FROM LOG WHERE id = 1 or 1"
    sql = "SELECT * FROM LOG WHERE id = %s"
    cursor.execute(sql, [log_id])
    
    while True:
        row = cursor.fetchone()
        if row is None:
            break
        print(row)

(1, '첫 번째 로그', 'chrome')


# SQL injection 공격의 예

In [60]:
column_data = {'message': '첫 로그', 'browser': 'chrome'}

with db_curser() as cursor:
    log_id = '1 or 1'
#     sql = "SELECT * FROM LOG WHERE id = " + str(log_id)
#     sql = "SELECT * FROM LOG WHERE id = 1 or 1"
    sql = "SELECT * FROM LOG WHERE id = %s"
    cursor.execute(sql, [log_id])
    for row in cursor:
        print(row)

(1, '첫 번째 로그', 'chrome')


# 데이터 매핑의 예

In [63]:
with db_curser() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = cursor.fetchall()
#    for row in cursor:
#        row[0]  # id
#        row[1]  # message 
#        row[2]  # chrome
rows_data

((1, '첫 번째 로그', 'chrome'), (2, '두 번째 로그', 'chrome'))

# dict 으로의 변환

# namedtuple

In [66]:
rows_data

In [79]:
from collections import namedtuple

Log = namedtuple('Log','id message browser')


In [80]:
log = Log(1, 'message', 'chrome')
print(log[0])
print(log[1])
print(log[2])

1
message
chrome


In [81]:
log.id, log.message, log.browser

(1, 'message', 'chrome')

In [82]:
with db_curser() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = [
        Log(*row)
        for row in cursor.fetchall()
        ]

rows_data

[Log(id=1, message='첫 번째 로그', browser='chrome'),
 Log(id=2, message='두 번째 로그', browser='chrome')]